In [ ]:

from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.data.models.bars import Bar

from config import ALPACA_SECRET_KEY, ALPACA_PUBLIC_KEY, FMP_API_KEY
from urllib.request import urlopen
from datetime import datetime, timezone, timedelta

import talib as ta
import numpy as np
import pandas as pd
import logging
import json
import pickle




```
$ tar -xzf ta-lib-0.6.4-src.tar.gz
$ cd ta-lib-0.6.4/
$ ./configure --prefix=/usr
$ make
$ sudo make install
```

next move the folder into your site packages folder, then do

```
python3 -m pip install TA-lib
```

In [3]:
url = f"https://financialmodelingprep.com/api/v3/nasdaq_constituent?apikey={FMP_API_KEY}"
response = urlopen(url)
data = response.read().decode("utf-8")
data = json.loads(data)

data

[{'symbol': 'ADBE',
  'name': 'Adobe Inc.',
  'sector': 'Technology',
  'subSector': 'Software - Infrastructure',
  'headQuarter': 'San Jose, CA',
  'dateFirstAdded': None,
  'cik': '0000796343',
  'founded': '1982-12-01'},
 {'symbol': 'AMAT',
  'name': 'Applied Materials',
  'sector': 'Technology',
  'subSector': 'Semiconductors',
  'headQuarter': 'Santa Clara, CA',
  'dateFirstAdded': None,
  'cik': '0000006951',
  'founded': '1967-11-10'},
 {'symbol': 'CSCO',
  'name': 'Cisco',
  'sector': 'Technology',
  'subSector': 'Communication Equipment',
  'headQuarter': 'San Jose, CA',
  'dateFirstAdded': None,
  'cik': '0000858877',
  'founded': '1984-12-10'},
 {'symbol': 'FAST',
  'name': 'Fastenal',
  'sector': 'Industrials',
  'subSector': 'Industrial - Distribution',
  'headQuarter': 'Winona, MN',
  'dateFirstAdded': None,
  'cik': '0000815556',
  'founded': '1967-01-01'},
 {'symbol': 'MSFT',
  'name': 'Microsoft',
  'sector': 'Technology',
  'subSector': 'Software - Infrastructure',
  

In [4]:
# Save the data as a pickle file
with open("nasdaq_data.pkl", "wb") as file:
    pickle.dump(data, file)

In [5]:
# Save the data as a pickle file
with open("nasdaq_data.pkl", "rb") as file:
    data = pickle.load(file)

In [6]:
NDQ100 = []
for element in data:
    NDQ100.append(element["symbol"])

In [7]:
client = StockHistoricalDataClient(ALPACA_PUBLIC_KEY, ALPACA_SECRET_KEY)

# client.get_stock_latest_quote(params)
start_day = datetime.now(tz=timezone.utc) - timedelta(days=3)
end_day = datetime.now(tz=timezone.utc)

params = StockBarsRequest(symbol_or_symbols=NDQ100, start=start_day, timeframe=TimeFrame.Minute, limit=10000000)

data = client.get_stock_bars(params)

In [8]:
data_iter = iter(data)
(_, unwrapped_data) = next(data_iter)

In [14]:
dfs = {}
for ticker in NDQ100:
    # extracts a ticker from the list of 100
    bars = unwrapped_data[ticker]
    # Convert to DataFrame
    df = pd.DataFrame([bar.__dict__ for bar in bars])
    dfs[ticker] = df

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,AAPL,2025-03-07 12:43:00+00:00,234.5200,234.5200,234.52,234.5200,251.0,11.0,234.520000
1,AAPL,2025-03-07 12:44:00+00:00,234.5000,234.5000,234.45,234.4500,862.0,38.0,234.464844
2,AAPL,2025-03-07 12:45:00+00:00,234.4000,234.4000,234.40,234.4000,399.0,28.0,234.400000
3,AAPL,2025-03-07 12:46:00+00:00,234.3900,234.4500,234.39,234.4500,1480.0,14.0,234.403707
4,AAPL,2025-03-07 12:47:00+00:00,234.4000,234.4000,234.40,234.4000,746.0,12.0,234.400000
...,...,...,...,...,...,...,...,...,...
760,AAPL,2025-03-10 12:23:00+00:00,235.4000,235.4500,235.40,235.4500,5419.0,79.0,235.418210
761,AAPL,2025-03-10 12:24:00+00:00,235.4700,235.5523,235.45,235.5523,933.0,52.0,235.494363
762,AAPL,2025-03-10 12:25:00+00:00,235.4503,235.6000,235.43,235.6000,9159.0,84.0,235.518388
763,AAPL,2025-03-10 12:26:00+00:00,235.6000,235.6000,235.45,235.4500,4765.0,71.0,235.474877
